In [1]:
import gc
import torch
from unsloth import FastLanguageModel

import transformers
from datasets import load_dataset
from peft import LoraConfig
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
import bitsandbytes as bnb

from tqdm import tqdm
import numpy as np
import warnings


warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 17:28:08 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
max_seq_length = 2048
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 2811,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 15:00:59 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [3]:
dataset = load_dataset("5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca", split="train")

In [4]:
dataset, dataset[0]

(Dataset({
     features: ['conversations', 'id'],
     num_rows: 12697
 }),
 {'conversations': [{'from': 'human',
    'value': 'Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."'},
   {'from': 'gpt',
    'value': 'Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.'},
   {'from': 'human',
    'value': 'Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?'},
   {'from': 'gpt',
    'value': 'Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như s

## **SFT**

In [5]:
SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."

def convert_to_chat_format(conversations):
    messages = [{"role": "system", "content": SYS_INSTRUCT}]
    for msg in conversations:
        role = "user" if msg["from"] == "human" else "assistant"
        messages.append({"role": role, "content": msg["value"]})
    return messages

def format_prompt(example):
    messages = convert_to_chat_format(example["conversations"])
    return {
        "text": tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    }

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_seq_length,
    )

dataset = dataset.map(format_prompt, remove_columns=dataset.column_names)
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [6]:
print(dataset[0]['text'])

<|im_start|>system
Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt.<|im_end|>
<|im_start|>user
Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."<|im_end|>
<|im_start|>assistant
Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.<|im_end|>
<|im_start|>user
Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?<|im_end|>
<|im_start|>assistant
Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4. P

In [7]:
# data collator for causal LM
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [8]:
hyperparameters = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 100,
    "num_train_epochs": 1,
    "save_strategy": "steps",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_ratio": 0.05,
    "bf16": True,
    "report_to": "none",
}
MAX_LENGTH = 2048

In [9]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [10]:
SFT_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-multi-conversation"

In [11]:
sft_config = SFTConfig(
    **{ **hyperparameters, "output_dir":
       SFT_OUTPUT_DIR , "max_seq_length": MAX_LENGTH, "packing": True}
)


sft_trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
    data_collator=data_collator,
)

sft_trainer.train()

Converting train dataset to ChatML (num_proc=20):   0%|          | 0/12697 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=20):   0%|          | 0/12697 [00:00<?, ? examples/s]

Packing train dataset (num_proc=20):   0%|          | 0/12697 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,354 | Num Epochs = 1 | Total steps = 397
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 59,867,136/3,000,000,000 (2.00% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,1.357400
200,1.229200
300,1.207200


TrainOutput(global_step=397, training_loss=1.2482317677072674, metrics={'train_runtime': 4142.437, 'train_samples_per_second': 1.534, 'train_steps_per_second': 0.096, 'total_flos': 2.212532916804649e+17, 'train_loss': 1.2482317677072674})

In [45]:
torch.cuda.empty_cache()

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    './Qwen2.5-3B-Instruct-multi-conversation/checkpoint-397',
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype=torch.bfloat16,
)

==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model.push_to_hub_merged(
    "binhphap5/Qwen2.5-3B-Instruct-Chat-sft",
    commit_message="Merge lora weights to push to hub",
    token="???"
)

tokenizer.push_to_hub(
    "binhphap5/Qwen2.5-3B-Instruct-Chat-sft",
    commit_message="Push tokenizer to hub",
    token="???"
)

In [50]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=128,
    temperature=0.8,
    top_p=0.95,
    top_k=10,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

prompt = [
    {"role": "system", "content": (
        "Bạn là một trợ lý AI thân thiện, "
        "hãy trả lời bằng tiếng Việt."
    )},
    {"role": "user", "content": (
        "Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "
        "\"Trận đấu là một thất bại nặng nề "
        "mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần.\""
    )},
    {"role": "assistant", "content": (
        "Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề."
    )},
    {"role": "user", "content": (
        "Bạn có thể đề xuất một số chiến lược mà "
        "nhóm có thể sử dụng để cải thiện hiệu suất "
        "của họ trong trận đấu tiếp theo không?"
    )}
]

chat_text = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    tokenize=False
)

inputs = tokenizer(
    chat_text,
    return_tensors="pt"
).to("cuda:0")

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        generation_config=generation_config,
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

if "assistant\n" in output_text:
    answer = output_text.split("assistant\n")[-1].strip()
else:
    answer = output_text.strip()

print("Assistant reply:", answer)

Assistant reply: Chắc chắn! Dưới đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo: 1. Xác định điểm yếu: Đánh giá lại trận đấu và xác định những điểm yếu của nhóm. Điều này có thể giúp nhóm xác định những điểm cần cải thiện và tập trung vào những điểm đó trong trận đấu tiếp theo. 2. Tăng cường tập luyện: Đặt ra mục tiêu tập luyện cụ thể và tập trung vào những kỹ năng cụ thể mà nhóm cần cải thiện. Điều này có thể bao gồm việc tập luyện


## **DPO**

In [2]:
dataset = load_dataset("thainq107/Vi-Alpaca-Preference")

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    "binhphap5/Qwen2.5-3B-Instruct-Chat-sft",
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype=torch.bfloat16,
)
lora_rank = 16
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 2811,
)

==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.4.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

In [5]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset.map(convert_to_conversational_preference_format)


In [15]:
def get_model_response(model, tokenizer, instruction):
    cur_conversation = [
        {"role": "system", "content": "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."},
        {"role": "user", "content": instruction}
    ]
    cur_input_prompt = tokenizer.apply_chat_template(
        cur_conversation, add_generation_prompt=True, tokenize=True
    )
    cur_output_ids = model.generate(
        input_ids=torch.LongTensor([cur_input_prompt]).to(model.device),
        attention_mask=torch.ones(1, len(cur_input_prompt)).to(model.device),
        max_new_tokens=1000,
        pad_token_id=tokenizer.eos_token_id,
    )
    cur_generated_ids = cur_output_ids[0][len(cur_input_prompt):]
    return tokenizer.decode(cur_generated_ids, skip_special_tokens=True)

instruction = "Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'."
response = get_model_response(model, tokenizer, instruction)
print(f"Instruction: {instruction}")
print(f"Response: {response}")

Instruction: Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'.
Response: Tạo meme với câu "thời trang muộn" không khó, nhưng tôi không thể tạo hình ảnh hay video meme được vì tôi là trợ lý AI dựa trên văn bản. Tuy nhiên, tôi có thể mô tả cho bạn một meme có thể làm từ cụm từ này:

Meme: 
- Hình nền là một người mặc quần áo thời trang hiện đại (ví dụ: áo khoác da, quần skinny jeans, giày cao gót).
- Trong góc trái của hình nền, xuất hiện một người khác đang mặc đồ thời trang thập niên 90 (ví dụ: áo sơ mi rộng, quần jean looser, giày cao cổ).
- Câu meme "Thời trang muộn" được in to và nổi bật ở giữa hai người.

Meme này sẽ nói lên ý nghĩa rằng mặc thời trang muộn tức là mặc những thứ đã lỗi thời so với xu hướng hiện tại. 

Bạn có muốn tôi vẽ ra hình ảnh này không? Nếu có, tôi sẽ cố gắng thực hiện. 

(Nếu không, đây là ví dụ meme mà bạn có thể tạo ra: "Thời trang muộn - Đừng mặc như người trước bạn").


In [6]:
dpo_dataset['train'][1000]

{'id': 'alpaca-14472',
 'question': 'Tạo một meme bằng cách sử dụng cụm từ "thời trang muộn".',
 'chosen': [{'content': '[Thêm hình ảnh người nổi tiếng mặc đồ lỗi thời, nhìn chẳng đẹp mắt, phù hợp với ý tưởng của meme] \n\nHãy tránh thời trang muộn, vì đôi khi nó có thể là điều tồi tệ nhất mà bạn có thể làm cho bản thân mình.',
   'role': 'assistant'}],
 'rejected': [{'content': 'Xin lỗi, nhưng vì tôi là một trí tuệ nhân tạo và không có khả năng thực hiện các tác vụ đồ họa, tôi không thể tạo ra một thiết kế áo phông cho bạn. Tuy nhiên, bạn có thể tham khảo các trang web thiết kế áo phông trực tuyến như Custom Ink, Printful hoặc Teespring để tạo ra thiết kế áo phông của riêng bạn dựa trên tiểu bang hoặc quốc gia của bạn. Chúc may mắn!',
   'role': 'assistant'}],
 'prompt': [{'content': 'Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt.',
   'role': 'system'},
  {'content': 'Tạo một meme bằng cách sử dụng cụm từ "thời trang muộn".',
   'role': 'user'}]}

In [7]:
hyperparameters = {
    "per_device_train_batch_size": 3,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 100,
    "max_steps": 500,
    "save_strategy": "steps",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_ratio": 0.05,
    "bf16": True,
}
MAX_LENGTH = 512

In [8]:
# Use wandb
import wandb
wandb.init(
    project="vi-alpaca-preference",
    name="Qwen2.5-3B-4bit-chat-dpo"
)


wandb: Currently logged in as: tridungluong123 (tridungluong123-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
DPO_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-chat-dpo"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_LENGTH }
)

dpo_trainer = DPOTrainer(
    model,
    ref_model = None,
    args=dpo_args,
    train_dataset=dpo_dataset['train'],
    processing_class=tokenizer,
)
dpo_trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65,017 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 8 x 1) = 24
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
100,0.353300,-1.060731,-4.226553,0.787500,3.165823,-211.543243,-358.595703,nan,nan,0,0,0,0
200,0.194700,-2.453786,-10.323666,0.874583,7.869880,-227.215591,-413.576630,nan,-3.680350,No Log,No Log,No Log,No Log
300,0.174300,-2.679446,-11.215009,0.884583,8.535563,-234.342728,-424.946289,nan,nan,No Log,No Log,No Log,No Log
400,0.176000,-2.190362,-11.647231,0.886250,9.456869,-230.944473,-433.805481,nan,nan,No Log,No Log,No Log,No Log
500,0.162100,-2.011829,-12.378806,0.889583,10.366978,-227.305801,-440.325043,nan,nan,No Log,No Log,No Log,No Log


TrainOutput(global_step=500, training_loss=0.2120833625793457, metrics={'train_runtime': 4202.3252, 'train_samples_per_second': 2.856, 'train_steps_per_second': 0.119, 'total_flos': 0.0, 'train_loss': 0.2120833625793457, 'epoch': 0.18456143588797122})

In [ ]:
torch.cuda.empty_cache()

In [18]:
instruction = "Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'."
response = get_model_response(model, tokenizer, instruction)
print(f"Instruction: {instruction}")
print(f"Response: {response}")

Instruction: Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'.
Response: Tạo meme với cụm từ 'thời trang muộn' không phải là khả thi bằng lời văn đơn giản vì meme thường cần hình ảnh hoặc video. Tuy nhiên, tôi có thể tạo ra một meme bằng cách sử dụng câu này:

[Ảnh: Một người mặc đồ thời trang cũ, nhưng đang đứng trước gương, phía sau là dòng chữ "Thời trang muộn".]

Meme này có thể sẽ gây cười khi thấy một người cố gắng cập nhật thời trang nhưng đã quá muộn so với xu hướng hiện tại. 

Hoặc nếu bạn muốn một meme ngắn gọn hơn, có thể như sau: 
[Ảnh: Một người phụ nữ đang thử quần áo mới, phía sau là dòng chữ "Thời trang muộn" với hình ảnh của một người đàn ông đang ăn bánh ngọt.] 

Trong cả hai meme này, câu "thời trang muộn" được dùng để tạo ra sự khác biệt và tạo ra một ý tưởng hài hước. 

Nhưng nhớ rằng meme tốt nhất thường dựa trên một hình ảnh hoặc video, nên nếu bạn muốn tạo meme, bạn nên tìm kiếm một hình ảnh phù hợp.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    './Qwen2.5-3B-Instruct-chat-dpo/checkpoint-500',
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype=torch.bfloat16,
)

model.push_to_hub_merged(
    "binhphap5/Qwen2.5-3B-Instruct-Chat-RLHF",
    commit_message="Merge lora weights to push to hub",
    token="???"
)

tokenizer.push_to_hub(
    "binhphap5/Qwen2.5-3B-Instruct-Chat-RLHF",
    commit_message="Push tokenizer to hub",
    token="???"
)

In [ ]:
"""
vllm serve binhphap5/Qwen2.5-3B-Instruct-Chat-RLHF \
--api-key binhphap5 \
--port 8000 \
--quantization bitsandbytes \
--enable-prefix-caching \
--swap-space 16 \
--gpu-memory-utilization 0.9 \
--disable-log-requests \
--max-model-len 2048 
"""